In [4]:
import os

import cv2
import torch
from torch.utils.data import DataLoader
import torch.optim as optim
from torch.nn import CTCLoss

from dataset import Synth90kDataset, synth90k_collate_fn
from model import CRNN
from evaluate import evaluate
from config import train_config as config
import numpy as np

import torch
import torch.nn as nn
import torchvision.transforms as transforms
from torchvision import datasets
from torch.utils.data import DataLoader
import timm

def train_batch(crnn, data, optimizer, criterion, device):
    crnn.train()
    images, targets, target_lengths = [d.to(device) for d in data]
    #print('image size')
    #print(images.size())
    logits = crnn(images)
    
    log_probs = torch.nn.functional.log_softmax(logits, dim=2)
    
    batch_size = images.size(0)
    input_lengths = torch.LongTensor([logits.size(0)] * batch_size)
    target_lengths = torch.flatten(target_lengths)
    '''print('batch logits log_probs targets input_len target_len')
    print(batch_size)
    print(logits.size())
    print(log_probs.size())
    print(targets.size())
    print(input_lengths.size())
    print(target_lengths.size())
    print('end')'''
    loss = criterion(log_probs, targets, input_lengths, target_lengths)

    optimizer.zero_grad()
    loss.backward()
    torch.nn.utils.clip_grad_norm_(crnn.parameters(), 5) # gradient clipping with 5
    optimizer.step()
    return loss.item()



In [2]:

epochs = config['epochs']
train_batch_size = config['train_batch_size']
eval_batch_size = config['eval_batch_size']
lr = config['lr']
show_interval = config['show_interval']
valid_interval = config['valid_interval']
save_interval = config['save_interval']
end_interval = 40000
cpu_workers = config['cpu_workers']
reload_checkpoint = config['reload_checkpoint']
valid_max_iter = config['valid_max_iter']

img_width = config['img_width']
img_height = config['img_height']
data_dir = config['data_dir']

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'device: {device}')

valid_dataset = Synth90kDataset(root_dir=data_dir, mode='test',
                                img_height=img_height, img_width=img_width)
valid_dataset.texts

device: cuda
0
00
002101
01
01206
01206368166
02110
0800
0th
1
10
100
101
1026
106
11
111
114
12
120
122
1229
13
13th
14
1410
14th
15
1600
1601
1620
165
1668
17
1701
1715
1770
17th
180
180plus
18th
19
191
1946
195
1957
1996
1st
1th
2
20
2000
2002
2008
2009
2010
2011
2076
20p
2101
212
21st
22
220
2204
222
22nd
236
25
250
25th
26
28
29th
2c
2gb
2nd
2th
3
30
31
311
319
31st
32803
32nd
33
34
358
35th
36
360
368166
3rd
3th
4
439
4447
45th
48245
4gb
4th
4u2watch
4x4
5
50
500
507
511
52
527
53
54
547
55
557
571
5742z
59
5a
5b
5th
6
60
61
617
61to69
631
69
6th
7
705
75
758
794447
794455
7th
8
80
800
801
818
83
88
89
8th
9
900
905
908
911
95
99
9th
a
a11
a12
a133
aa
aaa
aachen
aae
aah
aaliyah
aardvark
aardvarks
aaron
ab
aba
aback
abacus
abacuses
abaft
abalone
abalones
abandon
abandoned
abandoning
abandonment
abandons
abase
abased
abasement
abases
abash
abashed
abashedly
abashes
abashing
abashment
abasing
abate
abated
abatement
abates
abating
abattoir
abattoirs
abbas
abbasid
abbaye
abbe
abbes
ab

['slinking',
 'remodelers',
 'chronographs',
 'impeaching',
 'discombobulated',
 'loots',
 'underpays',
 'flameproofed',
 'enticements',
 'ukase',
 'descanted',
 'coolidge',
 'doziness',
 'deliberations',
 'supremacy',
 'remodelers',
 'resoled',
 'percipience',
 'terrifyingly',
 'prizewinner',
 'elaborately',
 'hopscotched',
 'telnetted',
 'disproportional',
 'warming',
 'bookstores',
 'meter',
 'savageness',
 'poppadoms',
 'guises',
 'tbs',
 'receptacle',
 'mas',
 'gladioli',
 'gunships',
 'coxes',
 'dozy',
 'empowered',
 'puzzlers',
 'designating',
 'goldmine',
 'wrist',
 'prepped',
 'punchy',
 'belching',
 'tuber',
 'lavatorial',
 'hostesses',
 'bathtub',
 'halfway',
 'rearm',
 'blather',
 'schoolboy',
 'townes',
 'annotating',
 'histories',
 'glossies',
 'rationalizes',
 'remaster',
 'yourself',
 'pot',
 'waffled',
 'darth',
 'crestless',
 'finn',
 'analogies',
 'fatsos',
 'penises',
 'domesticates',
 'brews',
 'ruthenium',
 'surer',
 'offenses',
 'sirrah',
 'stomping',
 'sparsity'

In [2]:
import torch
torch.cuda.is_available()

True

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'device: {device}')

device: cuda


# original CRNN

In [9]:
num_class = len(Synth90kDataset.LABEL2CHAR) + 1
print(num_class)
model = CRNN(1, 32, 100, num_class,
            map_to_seq_hidden=config['map_to_seq_hidden'],
            rnn_hidden=config['rnn_hidden'],
            leaky_relu=config['leaky_relu'])
#model.load_state_dict(torch.load('D:/lab2/深度学习课程设计/crnn-pytorch-master/checkpoints/mobileVit_chars74k_2.pth'),strict=False)
dummy_input = torch.randn(5, 1, 32, 100)
dummy_output =model(dummy_input)
dummy_output.size()

37
torch.Size([24, 5, 512])


torch.Size([24, 5, 37])

In [4]:


epochs = config['epochs']
train_batch_size = config['train_batch_size']
eval_batch_size = config['eval_batch_size']
lr = config['lr']
show_interval = config['show_interval']
valid_interval = config['valid_interval']
save_interval = config['save_interval']
end_interval = 40000
cpu_workers = config['cpu_workers']
reload_checkpoint = config['reload_checkpoint']
valid_max_iter = config['valid_max_iter']

img_width = config['img_width']
img_height = config['img_height']
data_dir = config['data_dir']

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'device: {device}')

train_dataset = Synth90kDataset(root_dir=data_dir, mode='train',
                                img_height=img_height, img_width=img_width)
valid_dataset = Synth90kDataset(root_dir=data_dir, mode='dev',
                                img_height=img_height, img_width=img_width)

train_loader = DataLoader(
    dataset=train_dataset,
    batch_size=train_batch_size,
    shuffle=True,
    num_workers=0,
    collate_fn=synth90k_collate_fn)
valid_loader = DataLoader(
    dataset=valid_dataset,
    batch_size=eval_batch_size,
    shuffle=True,
    num_workers=0,
    collate_fn=synth90k_collate_fn)

num_class = len(Synth90kDataset.LABEL2CHAR) + 1
crnn = CRNN(1, img_height, img_width, num_class,
            map_to_seq_hidden=config['map_to_seq_hidden'],
            rnn_hidden=config['rnn_hidden'],
            leaky_relu=config['leaky_relu'])
if reload_checkpoint:
    crnn.load_state_dict(torch.load(reload_checkpoint, map_location=device))
crnn.to(device)

optimizer = optim.Adam(crnn.parameters())
criterion = CTCLoss(reduction='sum', zero_infinity=True)
criterion.to(device)

loss_history = []
valid_history = []


assert save_interval % valid_interval == 0
i = 1
for epoch in range(1, epochs + 1):
    print(f'epoch: {epoch}')
    tot_train_loss = 0.
    tot_train_count = 0
    for train_data in train_loader:
        loss = train_batch(crnn, train_data, optimizer, criterion, device)
        train_size = train_data[0].size(0)

        tot_train_loss += loss
        tot_train_count += train_size
        if i % show_interval == 0:
            print('train_batch_loss[', i, ']: ', loss / train_size)
            loss_history.append(loss / train_size)

        if i % valid_interval == 0:
            evaluation = evaluate(crnn, valid_loader, criterion,
                                  decode_method=config['decode_method'],
                                  beam_size=config['beam_size'])
            print('valid_evaluation: loss={loss}, acc={acc}'.format(**evaluation))
            valid_history.append(evaluation)

            if i % save_interval == 0:
                prefix = 'crnn'
                loss = evaluation['loss']
                save_model_path = os.path.join(config['checkpoints_dir'],
                                               f'{prefix}_{i:06}_loss{loss}.pt')
                torch.save(crnn.state_dict(), save_model_path)
                print('save model at ', save_model_path)
        if i % end_interval == 0:
            break
            
        i += 1

    print('train_loss: ', tot_train_loss / tot_train_count)




device: cuda
epoch: 1
train_batch_loss[ 10 ]:  28.225263595581055
train_batch_loss[ 20 ]:  26.03909683227539
train_batch_loss[ 30 ]:  26.25438690185547
train_batch_loss[ 40 ]:  24.54315757751465
train_batch_loss[ 50 ]:  24.86879539489746
train_batch_loss[ 60 ]:  25.75336456298828
train_batch_loss[ 70 ]:  25.201152801513672
train_batch_loss[ 80 ]:  24.07999038696289
train_batch_loss[ 90 ]:  24.856430053710938
train_batch_loss[ 100 ]:  25.97860336303711
train_batch_loss[ 110 ]:  24.96133804321289
train_batch_loss[ 120 ]:  25.163803100585938
train_batch_loss[ 130 ]:  25.19082260131836
train_batch_loss[ 140 ]:  25.744182586669922
train_batch_loss[ 150 ]:  23.48008918762207
train_batch_loss[ 160 ]:  25.617610931396484
train_batch_loss[ 170 ]:  24.29474639892578
train_batch_loss[ 180 ]:  24.26898956298828
train_batch_loss[ 190 ]:  24.93378448486328
train_batch_loss[ 200 ]:  23.547264099121094
train_batch_loss[ 210 ]:  24.399791717529297
train_batch_loss[ 220 ]:  24.572406768798828
train_batc

train_batch_loss[ 1830 ]:  7.729409694671631
train_batch_loss[ 1840 ]:  7.510095119476318
train_batch_loss[ 1850 ]:  6.580531120300293
train_batch_loss[ 1860 ]:  5.414301872253418
train_batch_loss[ 1870 ]:  7.960390090942383
train_batch_loss[ 1880 ]:  6.4452667236328125
train_batch_loss[ 1890 ]:  6.497190475463867
train_batch_loss[ 1900 ]:  5.6955366134643555
train_batch_loss[ 1910 ]:  6.519881725311279
train_batch_loss[ 1920 ]:  5.64987850189209
train_batch_loss[ 1930 ]:  6.749014854431152
train_batch_loss[ 1940 ]:  6.6213483810424805
train_batch_loss[ 1950 ]:  7.094559669494629
train_batch_loss[ 1960 ]:  6.80831241607666
train_batch_loss[ 1970 ]:  6.873623847961426
train_batch_loss[ 1980 ]:  4.381885051727295
train_batch_loss[ 1990 ]:  6.039064407348633
train_batch_loss[ 2000 ]:  7.4137420654296875
train_batch_loss[ 2010 ]:  5.8848090171813965
train_batch_loss[ 2020 ]:  7.135617256164551
train_batch_loss[ 2030 ]:  8.15733528137207
train_batch_loss[ 2040 ]:  5.879508018493652
train_ba

train_batch_loss[ 3650 ]:  3.845104932785034
train_batch_loss[ 3660 ]:  3.4121932983398438
train_batch_loss[ 3670 ]:  3.1982998847961426
train_batch_loss[ 3680 ]:  3.077939987182617
train_batch_loss[ 3690 ]:  3.9100661277770996
train_batch_loss[ 3700 ]:  3.0077247619628906
train_batch_loss[ 3710 ]:  6.722925186157227
train_batch_loss[ 3720 ]:  3.780386447906494
train_batch_loss[ 3730 ]:  3.017224073410034
train_batch_loss[ 3740 ]:  2.735743522644043
train_batch_loss[ 3750 ]:  3.922070026397705
train_batch_loss[ 3760 ]:  2.70420503616333
train_batch_loss[ 3770 ]:  3.466705322265625
train_batch_loss[ 3780 ]:  2.820164680480957
train_batch_loss[ 3790 ]:  3.1432433128356934
train_batch_loss[ 3800 ]:  4.80106258392334
train_batch_loss[ 3810 ]:  5.483813762664795
train_batch_loss[ 3820 ]:  3.9595232009887695
train_batch_loss[ 3830 ]:  3.122011184692383
train_batch_loss[ 3840 ]:  2.8177084922790527
train_batch_loss[ 3850 ]:  3.9246699810028076
train_batch_loss[ 3860 ]:  1.4353463649749756
tra

Evaluate:   8%|█████▊                                                               | 132/1568 [01:26<15:19,  1.56it/s]

Corrupted image for 467544


...

Corrupted image for 419552


Evaluate:  71%|████████████████████████████████████████████████▏                   | 1110/1568 [12:38<05:12,  1.46it/s]

Corrupted image for 467544


Evaluate: 100%|████████████████████████████████████████████████████████████████████| 1568/1568 [17:57<00:00,  1.46it/s]

valid_evaluation: loss=1.3993982520835508, acc=0.8302139936766102
save model at  D:/lab2/深度学习课程设计/crnn-pytorch-master/checkpoints/crnn_040000_loss1.3993982520835508.pt
train_loss:  2.7786078856851906


In [5]:

import os
import pickle

histories = [loss_history,valid_history]
with open('D:/lab2/深度学习课程设计/crnn-pytorch-master/train_history/crnn_1', 'wb') as fp:
    pickle.dump(histories, fp)

In [6]:
with open ('D:/lab2/深度学习课程设计/crnn-pytorch-master/train_history/crnn_1', 'rb') as fp:
    temp = pickle.load(fp)
temp

[[28.225263595581055,
  26.03909683227539,
  26.25438690185547,
  24.54315757751465,
  24.86879539489746,
  25.75336456298828,
  25.201152801513672,
  24.07999038696289,
  24.856430053710938,
  25.97860336303711,
  24.96133804321289,
  25.163803100585938,
  25.19082260131836,
  25.744182586669922,
  23.48008918762207,
  25.617610931396484,
  24.29474639892578,
  24.26898956298828,
  24.93378448486328,
  23.547264099121094,
  24.399791717529297,
  24.572406768798828,
  23.88490104675293,
  25.667999267578125,
  25.35882568359375,
  25.277908325195312,
  25.746557235717773,
  22.447723388671875,
  23.455978393554688,
  22.873191833496094,
  25.201431274414062,
  24.343748092651367,
  24.091259002685547,
  24.95233154296875,
  25.224937438964844,
  25.846519470214844,
  25.47356414794922,
  23.613039016723633,
  23.92202377319336,
  24.313255310058594,
  24.96070098876953,
  23.272903442382812,
  23.322437286376953,
  25.530223846435547,
  24.166048049926758,
  23.35240936279297,
  24.282

In [7]:
pytorch_total_params = sum(p.numel() for p in crnn.parameters())
pytorch_total_params

7839077

# mobilevit-RNN 

In [10]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from torchvision import datasets
from torch.utils.data import DataLoader
import timm

class MobileVit_improv(nn.Module):
    def __init__(self,img_channel,  img_height, img_width,num_class,rnn_hidden=256):
        super(MobileVit_improv, self).__init__()
        self.model = timm.create_model('mobilevit_s.cvnets_in1k', pretrained=False,features_only=True)
        self.model.stem.conv = nn.Conv2d(1, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        
        map_to_seq_hidden = 1000
        #self.model.head.fc = nn.Linear(self.model.head.in_features, map_to_seq)
        output_channel, output_height, output_width = self._compute_output_shape(img_channel, img_height, img_width)
        
        self.map_to_seq = nn.Linear(output_channel * output_height, map_to_seq_hidden)
        self.rnn1 = nn.LSTM(map_to_seq_hidden, rnn_hidden, bidirectional=True)
        self.rnn2 = nn.LSTM(2 * rnn_hidden, rnn_hidden, bidirectional=True)

        self.dense = nn.Linear(2 * rnn_hidden, num_class)
        
    
    
    def forward(self, x):
        x = self.model(x)[2]
        #print(x)
        batch, channel, height, width = x.size()
        #print('batch, channel, height, width')
        #print(batch, channel, height, width)
        x = x.view(batch, channel * height, width)
        x = x.permute(2, 0, 1)  # (width, batch, feature)
        #print(x.size())
        seq = self.map_to_seq(x)

        recurrent, _ = self.rnn1(seq)
        recurrent, _ = self.rnn2(recurrent)

        output = self.dense(recurrent)
        return output  # shape: (seq_len, batch, num_class)
        

    def _compute_output_shape(self,img_channel,  img_height, img_width):
        dummy_input = torch.randn(1, img_channel, img_height, img_width)
        dummy_output = self.model(dummy_input)
        #print(len(dummy_output))
        
        _, channels, height, width = dummy_output[2].size()
        #print('hihi')
        #print(channels, height, width)
        return channels, height, width

In [11]:
num_class = len(Synth90kDataset.LABEL2CHAR) + 1
model = MobileVit_improv(1,32,100,num_class)
#model.load_state_dict(torch.load('D:/lab2/深度学习课程设计/crnn-pytorch-master/checkpoints/mobileVit_chars74k_2.pth'),strict=False)
dummy_input = torch.randn(5, 1, 32, 100)
dummy_output =model(dummy_input)
dummy_output.size()

torch.Size([13, 5, 384])


torch.Size([13, 5, 37])

In [9]:
pytorch_total_params = sum(p.numel() for p in model.parameters())
pytorch_total_params

9494669

In [10]:

epochs = config['epochs']
train_batch_size = config['train_batch_size']
eval_batch_size = config['eval_batch_size']
lr = config['lr']
show_interval = config['show_interval']
valid_interval = config['valid_interval']
save_interval = config['save_interval']
end_interval = 40000
cpu_workers = config['cpu_workers']
reload_checkpoint = config['reload_checkpoint']
valid_max_iter = config['valid_max_iter']

img_width = config['img_width']
img_height = config['img_height']
img_channel = config['img_channel']
data_dir = config['data_dir']
#print(img_width,img_height)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'device: {device}')

train_dataset = Synth90kDataset(root_dir=data_dir, mode='train',
                                img_height=img_height, img_width=img_width)
valid_dataset = Synth90kDataset(root_dir=data_dir, mode='dev',
                                img_height=img_height, img_width=img_width)

print(valid_dataset.__len__())
train_loader = DataLoader(
    dataset=train_dataset,
    batch_size=train_batch_size,
    shuffle=True,
    num_workers=0,
    collate_fn=synth90k_collate_fn)
valid_loader = DataLoader(
    dataset=valid_dataset,
    batch_size=eval_batch_size,
    shuffle=True,
    num_workers=0,
    collate_fn=synth90k_collate_fn)

num_class = len(Synth90kDataset.LABEL2CHAR) + 1
'''
crnn = CRNN(1, img_height, img_width, num_class,
            map_to_seq_hidden=config['map_to_seq_hidden'],
            rnn_hidden=config['rnn_hidden'],
            leaky_relu=config['leaky_relu'])
if reload_checkpoint:
    crnn.load_state_dict(torch.load(reload_checkpoint, map_location=device))
'''  
    
crnn = MobileVit_improv(1,32,100,num_class)
#crnn.load_state_dict(torch.load('D:/lab2/深度学习课程设计/crnn-pytorch-master/checkpoints/mobileVit_chars74k_2.pth'),strict=False)


crnn.to(device)

optimizer = optim.Adam(crnn.parameters())
criterion = CTCLoss(reduction='sum', zero_infinity=True)
criterion.to(device)
loss_history=[]
valid_history=[]
assert save_interval % valid_interval == 0
i = 1
for epoch in range(1, epochs + 1):
    print(f'epoch: {epoch}')
    tot_train_loss = 0.
    tot_train_count = 0
    for train_data in train_loader:
        loss = train_batch(crnn, train_data, optimizer, criterion, device)
        train_size = train_data[0].size(0)

        tot_train_loss += loss
        tot_train_count += train_size
        if i % show_interval == 0:
            print('train_batch_loss[', i, ']: ', loss / train_size)
            loss_history.append(loss / train_size)

        if i % valid_interval == 0:
            evaluation = evaluate(crnn, valid_loader, criterion,
                                  decode_method=config['decode_method'],
                                  beam_size=config['beam_size'])
            print('valid_evaluation: loss={loss}, acc={acc}'.format(**evaluation))
            valid_history.append(evaluation)
            if i % save_interval == 0:
                prefix = 'mobilevit_rnn'
                loss = evaluation['loss']
                save_model_path = os.path.join(config['checkpoints_dir'],
                                               f'{prefix}_{i:06}_loss{loss}.pt')
                torch.save(crnn.state_dict(), save_model_path)
                print('save model at ', save_model_path)

        if i % end_interval == 0:
            break
        i += 1

    print('train_loss: ', tot_train_loss / tot_train_count)

device: cuda
802734
epoch: 1
train_batch_loss[ 10 ]:  24.15996551513672
train_batch_loss[ 20 ]:  23.147846221923828
train_batch_loss[ 30 ]:  25.26816177368164
train_batch_loss[ 40 ]:  23.757020950317383
train_batch_loss[ 50 ]:  22.980716705322266
train_batch_loss[ 60 ]:  24.600717544555664
train_batch_loss[ 70 ]:  23.73973274230957
train_batch_loss[ 80 ]:  23.625492095947266
train_batch_loss[ 90 ]:  25.49618911743164
train_batch_loss[ 100 ]:  23.96426010131836
train_batch_loss[ 110 ]:  23.72730255126953
train_batch_loss[ 120 ]:  23.053531646728516
train_batch_loss[ 130 ]:  24.715412139892578
train_batch_loss[ 140 ]:  24.980304718017578
train_batch_loss[ 150 ]:  24.164417266845703
train_batch_loss[ 160 ]:  24.762134552001953
train_batch_loss[ 170 ]:  22.56833839416504
train_batch_loss[ 180 ]:  23.74498748779297
train_batch_loss[ 190 ]:  23.180675506591797
train_batch_loss[ 200 ]:  23.6967830657959
train_batch_loss[ 210 ]:  22.54412841796875
train_batch_loss[ 220 ]:  23.37295150756836
tr

train_batch_loss[ 1830 ]:  6.777268886566162
train_batch_loss[ 1840 ]:  5.645070552825928
train_batch_loss[ 1850 ]:  6.551827907562256
train_batch_loss[ 1860 ]:  8.61594009399414
train_batch_loss[ 1870 ]:  7.092624664306641
train_batch_loss[ 1880 ]:  5.739577293395996
train_batch_loss[ 1890 ]:  5.780702590942383
train_batch_loss[ 1900 ]:  5.878946781158447
train_batch_loss[ 1910 ]:  6.097238540649414
train_batch_loss[ 1920 ]:  6.918108940124512
train_batch_loss[ 1930 ]:  8.439742088317871
train_batch_loss[ 1940 ]:  6.271666526794434
train_batch_loss[ 1950 ]:  7.155298709869385
train_batch_loss[ 1960 ]:  6.4985575675964355
train_batch_loss[ 1970 ]:  7.308804512023926
train_batch_loss[ 1980 ]:  6.367013454437256
train_batch_loss[ 1990 ]:  4.788702011108398
train_batch_loss[ 2000 ]:  6.129495620727539
train_batch_loss[ 2010 ]:  5.942590713500977
train_batch_loss[ 2020 ]:  6.439490795135498
train_batch_loss[ 2030 ]:  7.721747875213623
train_batch_loss[ 2040 ]:  4.685502052307129
train_batc

train_batch_loss[ 3660 ]:  4.474054336547852
train_batch_loss[ 3670 ]:  3.400106430053711
train_batch_loss[ 3680 ]:  3.6643457412719727
train_batch_loss[ 3690 ]:  6.028110027313232
train_batch_loss[ 3700 ]:  5.409153938293457
train_batch_loss[ 3710 ]:  4.444601058959961
train_batch_loss[ 3720 ]:  5.254767417907715
train_batch_loss[ 3730 ]:  2.7112183570861816
train_batch_loss[ 3740 ]:  3.559403419494629
train_batch_loss[ 3750 ]:  4.480258464813232
train_batch_loss[ 3760 ]:  5.895580768585205
train_batch_loss[ 3770 ]:  3.7646400928497314
train_batch_loss[ 3780 ]:  5.0508012771606445
train_batch_loss[ 3790 ]:  4.66364860534668
train_batch_loss[ 3800 ]:  3.601590156555176
train_batch_loss[ 3810 ]:  4.4027605056762695
train_batch_loss[ 3820 ]:  5.053958415985107
train_batch_loss[ 3830 ]:  3.3217406272888184
train_batch_loss[ 3840 ]:  5.172924518585205
train_batch_loss[ 3850 ]:  4.680365562438965
train_batch_loss[ 3860 ]:  4.868142604827881
train_batch_loss[ 3870 ]:  5.086209774017334
train

Evaluate:   1%|█                                                                     | 23/1568 [00:17<19:02,  1.35it/s]

Corrupted image for 419552


...

Corrupted image for 467544


Evaluate:  90%|█████████████████████████████████████████████████████████████▍      | 1417/1568 [17:40<01:48,  1.39it/s]

Corrupted image for 419552


Evaluate: 100%|████████████████████████████████████████████████████████████████████| 1568/1568 [19:57<00:00,  1.31it/s]

valid_evaluation: loss=2.5035567038361, acc=0.6781125503591476
save model at  D:/lab2/深度学习课程设计/crnn-pytorch-master/checkpoints/mobilevit_rnn_040000_loss2.5035567038361.pt
train_loss:  3.421708146211505


In [12]:

import os
import pickle

 
with open('D:/lab2/深度学习课程设计/crnn-pytorch-master/train_history/mobileVit_rnn_loss', 'wb') as fp:
    pickle.dump([loss_history,valid_history], fp)

MemoryError: 

In [ ]:
with open ('D:/lab2/深度学习课程设计/crnn-pytorch-master/train_history/mobileVit_rnn_1', 'rb') as fp:
    temp = pickle.load(fp)
temp

In [ ]:
pytorch_total_params = sum(p.numel() for p in crnn.parameters())
pytorch_total_params

In [13]:
loss_history

[24.15996551513672,
 23.147846221923828,
 25.26816177368164,
 23.757020950317383,
 22.980716705322266,
 24.600717544555664,
 23.73973274230957,
 23.625492095947266,
 25.49618911743164,
 23.96426010131836,
 23.72730255126953,
 23.053531646728516,
 24.715412139892578,
 24.980304718017578,
 24.164417266845703,
 24.762134552001953,
 22.56833839416504,
 23.74498748779297,
 23.180675506591797,
 23.6967830657959,
 22.54412841796875,
 23.37295150756836,
 21.49481773376465,
 23.705638885498047,
 20.418201446533203,
 23.384031295776367,
 21.797086715698242,
 22.812301635742188,
 20.524314880371094,
 21.212459564208984,
 23.029254913330078,
 21.234966278076172,
 21.494609832763672,
 21.280935287475586,
 22.542755126953125,
 21.828720092773438,
 21.478775024414062,
 19.28566551208496,
 20.738784790039062,
 20.330886840820312,
 20.08675765991211,
 17.75045394897461,
 18.13392448425293,
 18.938695907592773,
 20.794984817504883,
 18.07672119140625,
 18.164703369140625,
 18.431640625,
 15.343786239624

# CRNN with MobileViT blocks

In [27]:
class CRNN_vit(nn.Module):

    def __init__(self, img_channel, img_height, img_width, num_class,
                 map_to_seq_hidden=64, rnn_hidden=256, leaky_relu=False):
        super(CRNN_vit, self).__init__()

        self.cnn, (output_channel, output_height, output_width) = \
            self._cnn_backbone(img_channel, img_height, img_width, leaky_relu)

        self.map_to_seq = nn.Linear(output_channel * output_height, map_to_seq_hidden)

        self.rnn1 = nn.LSTM(map_to_seq_hidden, rnn_hidden, bidirectional=True)
        self.rnn2 = nn.LSTM(2 * rnn_hidden, rnn_hidden, bidirectional=True)

        self.dense = nn.Linear(2 * rnn_hidden, num_class)

    def _cnn_backbone(self, img_channel, img_height, img_width, leaky_relu):
        assert img_height % 16 == 0
        assert img_width % 4 == 0

        channels = [img_channel, 64, 128, 256, 256, 512, 512, 512]
        kernel_sizes = [3, 3, 3, 3, 3, 3, 2]
        strides = [1, 1, 1, 1, 1, 1, 1]
        paddings = [1, 1, 1, 1, 1, 1, 0]
        L = [2, 4, 3]
        dims = [64, 80, 96]
        patch_size=(2,2)
        cnn = nn.Sequential()
        
        

        def conv_relu(i, batch_norm=False):
            # shape of input: (batch, input_channel, height, width)
            input_channel = channels[i]
            output_channel = channels[i+1]

            cnn.add_module(
                f'conv{i}',
                nn.Conv2d(input_channel, output_channel, kernel_sizes[i], strides[i], paddings[i])
            )

            if batch_norm:
                cnn.add_module(f'batchnorm{i}', nn.BatchNorm2d(output_channel))

            relu = nn.LeakyReLU(0.2, inplace=True) if leaky_relu else nn.ReLU(inplace=True)
            cnn.add_module(f'relu{i}', relu)

        # size of image: (channel, height, width) = (img_channel, img_height, img_width)
        conv_relu(0)
        cnn.add_module('pooling0', nn.MaxPool2d(kernel_size=2, stride=2))
        # (64, img_height // 2, img_width // 2)
        
        # add vit block
        cnn.add_module('MobViT1', MobileViTBlock(dims[0], L[0], channels[1], kernel_sizes[1], patch_size, int(dims[0]*2)))
        conv_relu(1)
        cnn.add_module('pooling1', nn.MaxPool2d(kernel_size=2, stride=2))
        # (128, img_height // 4, img_width // 4)
        
        # add vit block
        cnn.add_module('MobViT1', MobileViTBlock(dims[1], L[1], channels[1], kernel_sizes[2], patch_size, int(dims[1]*2)))
        conv_relu(2)
        conv_relu(3)
        cnn.add_module(
            'pooling2',
            nn.MaxPool2d(kernel_size=(2, 1))
        )  # (256, img_height // 8, img_width // 4)

        conv_relu(4, batch_norm=True)
        conv_relu(5, batch_norm=True)
        cnn.add_module(
            'pooling3',
            nn.MaxPool2d(kernel_size=(2, 1))
        )  # (512, img_height // 16, img_width // 4)

        conv_relu(6)  # (512, img_height // 16 - 1, img_width // 4 - 1)

        output_channel, output_height, output_width = \
            channels[-1], img_height // 16 - 1, img_width // 4 - 1
        return cnn, (output_channel, output_height, output_width)

    def forward(self, images):
        # shape of images: (batch, channel, height, width)

        conv = self.cnn(images)
        batch, channel, height, width = conv.size()
        
        conv = conv.view(batch, channel * height, width)
        conv = conv.permute(2, 0, 1)  # (width, batch, feature)
        #print(conv.size())
        seq = self.map_to_seq(conv)

        recurrent, _ = self.rnn1(seq)
        recurrent, _ = self.rnn2(recurrent)

        output = self.dense(recurrent)
        return output  # shape: (seq_len, batch, num_class)

In [28]:
num_class = len(Synth90kDataset.LABEL2CHAR) + 1
print(num_class)
model = CRNN_vit(1, 32, 100, num_class,
            map_to_seq_hidden=config['map_to_seq_hidden'],
            rnn_hidden=config['rnn_hidden'],
            leaky_relu=config['leaky_relu'])
#model.load_state_dict(torch.load('D:/lab2/深度学习课程设计/crnn-pytorch-master/checkpoints/mobileVit_chars74k_2.pth'),strict=False)
dummy_input = torch.randn(5, 1, 32, 100)
dummy_output =model(dummy_input)
dummy_output.size()

37


torch.Size([24, 5, 37])

In [29]:
pytorch_total_params = sum(p.numel() for p in model.parameters())
pytorch_total_params

8106373

In [31]:

epochs = config['epochs']
train_batch_size = config['train_batch_size']
eval_batch_size = config['eval_batch_size']
lr = config['lr']
show_interval = config['show_interval']
valid_interval = config['valid_interval']
save_interval = config['save_interval']
end_interval = 40000
cpu_workers = config['cpu_workers']
reload_checkpoint = config['reload_checkpoint']
valid_max_iter = config['valid_max_iter']

img_width = 128
img_height = 32
img_channel = config['img_channel']
data_dir = config['data_dir']
#print(img_width,img_height)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'device: {device}')

train_dataset = Synth90kDataset(root_dir=data_dir, mode='train',
                                img_height=img_height, img_width=img_width)
valid_dataset = Synth90kDataset(root_dir=data_dir, mode='dev',
                                img_height=img_height, img_width=img_width)

print(valid_dataset.__len__())
train_loader = DataLoader(
    dataset=train_dataset,
    batch_size=train_batch_size,
    shuffle=True,
    num_workers=0,
    collate_fn=synth90k_collate_fn)
valid_loader = DataLoader(
    dataset=valid_dataset,
    batch_size=eval_batch_size,
    shuffle=True,
    num_workers=0,
    collate_fn=synth90k_collate_fn)

num_class = len(Synth90kDataset.LABEL2CHAR) + 1
'''
crnn = CRNN(1, img_height, img_width, num_class,
            map_to_seq_hidden=config['map_to_seq_hidden'],
            rnn_hidden=config['rnn_hidden'],
            leaky_relu=config['leaky_relu'])
if reload_checkpoint:
    crnn.load_state_dict(torch.load(reload_checkpoint, map_location=device))
'''  
crnn = CRNN_vit(1, 32, 100, num_class,
            map_to_seq_hidden=config['map_to_seq_hidden'],
            rnn_hidden=config['rnn_hidden'],
            leaky_relu=config['leaky_relu'])

pytorch_total_params = sum(p.numel() for p in crnn.parameters())
print('total param: ', pytorch_total_params)


device: cuda
802734
total param:  8106373


In [33]:

crnn.to(device)

optimizer = optim.Adam(crnn.parameters())
criterion = CTCLoss(reduction='sum', zero_infinity=True)
criterion.to(device)
loss_history=[]
valid_history=[]
assert save_interval % valid_interval == 0
i = 1
for epoch in range(1, epochs + 1):
    print(f'epoch: {epoch}')
    tot_train_loss = 0.
    tot_train_count = 0
    for train_data in train_loader:
        loss = train_batch(crnn, train_data, optimizer, criterion, device)
        train_size = train_data[0].size(0)

        tot_train_loss += loss
        tot_train_count += train_size
        if i % show_interval == 0:
            print('train_batch_loss[', i, ']: ', loss / train_size)
            loss_history.append(loss / train_size)

        if i % valid_interval == 0:
            evaluation = evaluate(crnn, valid_loader, criterion,
                                  decode_method=config['decode_method'],
                                  beam_size=config['beam_size'])
            print('valid_evaluation: loss={loss}, acc={acc}'.format(**evaluation))
            valid_history.append(evaluation)
            if i % save_interval == 0:
                prefix = 'mobilevit_rnn_v4'
                loss = evaluation['loss']
                save_model_path = os.path.join(config['checkpoints_dir'],
                                               f'{prefix}_{i:06}_loss{loss}.pt')
                torch.save(crnn.state_dict(), save_model_path)
                print('save model at ', save_model_path)

        if i % end_interval == 0:
            break
        i += 1

    print('train_loss: ', tot_train_loss / tot_train_count)

epoch: 1
train_batch_loss[ 10 ]:  26.32318115234375
train_batch_loss[ 20 ]:  25.92038917541504
train_batch_loss[ 30 ]:  25.77983856201172
train_batch_loss[ 40 ]:  26.687671661376953
train_batch_loss[ 50 ]:  24.70000457763672
train_batch_loss[ 60 ]:  25.67837142944336
train_batch_loss[ 70 ]:  25.52066421508789
train_batch_loss[ 80 ]:  25.48587989807129
train_batch_loss[ 90 ]:  25.60915756225586
train_batch_loss[ 100 ]:  27.051166534423828
train_batch_loss[ 110 ]:  24.26970863342285
train_batch_loss[ 120 ]:  23.374496459960938
train_batch_loss[ 130 ]:  26.735599517822266
train_batch_loss[ 140 ]:  24.747140884399414
train_batch_loss[ 150 ]:  23.061668395996094
train_batch_loss[ 160 ]:  24.722734451293945
train_batch_loss[ 170 ]:  23.789003372192383
train_batch_loss[ 180 ]:  24.68138313293457
train_batch_loss[ 190 ]:  22.923885345458984
train_batch_loss[ 200 ]:  25.117904663085938
train_batch_loss[ 210 ]:  25.955337524414062
train_batch_loss[ 220 ]:  23.483539581298828
train_batch_loss[ 23

train_batch_loss[ 1830 ]:  11.268327713012695
train_batch_loss[ 1840 ]:  10.457213401794434
train_batch_loss[ 1850 ]:  12.321795463562012
train_batch_loss[ 1860 ]:  9.384542465209961
train_batch_loss[ 1870 ]:  6.521623134613037
train_batch_loss[ 1880 ]:  8.652471542358398
train_batch_loss[ 1890 ]:  8.335664749145508
train_batch_loss[ 1900 ]:  8.399188041687012
train_batch_loss[ 1910 ]:  8.04068374633789
train_batch_loss[ 1920 ]:  7.094365119934082
Corrupted image for 4436875
train_batch_loss[ 1930 ]:  7.1413397789001465
train_batch_loss[ 1940 ]:  8.44589614868164
train_batch_loss[ 1950 ]:  9.468132019042969
train_batch_loss[ 1960 ]:  8.121112823486328
train_batch_loss[ 1970 ]:  7.2706122398376465
train_batch_loss[ 1980 ]:  7.10850715637207
train_batch_loss[ 1990 ]:  10.292952537536621
train_batch_loss[ 2000 ]:  6.333601951599121
train_batch_loss[ 2010 ]:  7.355423450469971
train_batch_loss[ 2020 ]:  7.241215229034424
train_batch_loss[ 2030 ]:  8.5230131149292
train_batch_loss[ 2040 ]: 

train_batch_loss[ 3630 ]:  2.8169517517089844
train_batch_loss[ 3640 ]:  3.662916660308838
train_batch_loss[ 3650 ]:  2.7854199409484863
train_batch_loss[ 3660 ]:  2.286363363265991
train_batch_loss[ 3670 ]:  2.63492488861084
train_batch_loss[ 3680 ]:  4.022756099700928
train_batch_loss[ 3690 ]:  2.150639533996582
train_batch_loss[ 3700 ]:  2.204545259475708
train_batch_loss[ 3710 ]:  3.2525794506073
train_batch_loss[ 3720 ]:  2.9519951343536377
train_batch_loss[ 3730 ]:  3.031076192855835
train_batch_loss[ 3740 ]:  2.363663673400879
train_batch_loss[ 3750 ]:  3.9293055534362793
train_batch_loss[ 3760 ]:  3.7389748096466064
train_batch_loss[ 3770 ]:  2.056551456451416
train_batch_loss[ 3780 ]:  3.18585205078125
train_batch_loss[ 3790 ]:  2.531254529953003
train_batch_loss[ 3800 ]:  3.5934948921203613
train_batch_loss[ 3810 ]:  3.289252758026123
train_batch_loss[ 3820 ]:  2.209596872329712
train_batch_loss[ 3830 ]:  3.4715890884399414
train_batch_loss[ 3840 ]:  3.1296064853668213
train_

Evaluate:   1%|▎                                                                      | 8/1568 [00:11<37:35,  1.45s/it]

Corrupted image for 467544


...

Corrupted image for 620308


Evaluate:  60%|█████████████████████████████████████████▏                           | 935/1568 [21:57<14:58,  1.42s/it]

Corrupted image for 419552


Evaluate: 100%|████████████████████████████████████████████████████████████████████| 1568/1568 [36:57<00:00,  1.41s/it]

valid_evaluation: loss=0.9095548662792806, acc=0.8900955982928342
save model at  D:/lab2/深度学习课程设计/crnn-pytorch-master/checkpoints/mobilevit_rnn_v4_040000_loss0.9095548662792806.pt
train_loss:  2.3544757948610933


In [42]:
valid_history2 = []
for i in range(len(valid_history)):
    evaluation = {
        'loss': valid_history[i]['loss'],
        'acc': valid_history[i]['acc'],
    }
    valid_history2.append(evaluation)
valid_history2

[{'loss': 2.5910161001522902, 'acc': 0.7123492962799632},
 {'loss': 1.8641845750114072, 'acc': 0.7844603567308722},
 {'loss': 1.4699078681275808, 'acc': 0.8244325019246724},
 {'loss': 1.2746191259321709, 'acc': 0.8437028953551239},
 {'loss': 1.1683830385527212, 'acc': 0.859780450311062},
 {'loss': 1.0911384808359814, 'acc': 0.8690562502647203},
 {'loss': 1.046315777225071, 'acc': 0.8752089733336323},
 {'loss': 0.9620996700228386, 'acc': 0.8836451427247382},
 {'loss': 0.9482514772701548, 'acc': 0.8870435287405293},
 {'loss': 0.9095548662792806, 'acc': 0.8900955982928342}]

In [43]:

import os
import pickle

 
with open('D:/lab2/深度学习课程设计/crnn-pytorch-master/train_history/mobileVit_rnn_3_loss', 'wb') as fp:
    pickle.dump(loss_history, fp)
     
with open('D:/lab2/深度学习课程设计/crnn-pytorch-master/train_history/mobileVit_rnn_3_valid', 'wb') as fp:
    pickle.dump(valid_history2, fp)

In [44]:
with open ('D:/lab2/深度学习课程设计/crnn-pytorch-master/train_history/mobileVit_rnn_3_loss', 'rb') as fp:
    temp = pickle.load(fp)
temp

[26.32318115234375,
 25.92038917541504,
 25.77983856201172,
 26.687671661376953,
 24.70000457763672,
 25.67837142944336,
 25.52066421508789,
 25.48587989807129,
 25.60915756225586,
 27.051166534423828,
 24.26970863342285,
 23.374496459960938,
 26.735599517822266,
 24.747140884399414,
 23.061668395996094,
 24.722734451293945,
 23.789003372192383,
 24.68138313293457,
 22.923885345458984,
 25.117904663085938,
 25.955337524414062,
 23.483539581298828,
 24.428390502929688,
 25.73165512084961,
 23.33498191833496,
 24.1345157623291,
 25.130659103393555,
 24.80514907836914,
 26.1441650390625,
 23.992998123168945,
 24.118572235107422,
 25.045103073120117,
 24.433238983154297,
 24.65667724609375,
 23.566417694091797,
 23.89128875732422,
 25.205039978027344,
 23.609172821044922,
 23.55706024169922,
 23.761512756347656,
 22.978084564208984,
 23.42521858215332,
 23.377002716064453,
 26.37904930114746,
 24.741548538208008,
 22.946739196777344,
 23.64943504333496,
 23.830875396728516,
 22.28328704833

# mobilevit v2

In [ ]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from torchvision import datasets
from torch.utils.data import DataLoader
import timm

class MobileVit_improv(nn.Module):
    def __init__(self,img_channel,  img_height, img_width,num_class,rnn_hidden=256):
        super(MobileVit_improv, self).__init__()
        self.model = timm.create_model('mobilevit_s.cvnets_in1k', pretrained=False,features_only=True)
        self.model.stem.conv = nn.Conv2d(1, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        
        map_to_seq_hidden = 1000
        #self.model.head.fc = nn.Linear(self.model.head.in_features, map_to_seq)
        output_channel, output_height, output_width = self._compute_output_shape(img_channel, img_height, img_width)
        
        self.map_to_seq = nn.Linear(output_channel * output_height, map_to_seq_hidden)
        self.rnn1 = nn.LSTM(map_to_seq_hidden, rnn_hidden, bidirectional=True)
        self.rnn2 = nn.LSTM(2 * rnn_hidden, rnn_hidden, bidirectional=True)

        self.dense = nn.Linear(2 * rnn_hidden, num_class)
        
    
    
    def forward(self, x):
        x = self.model(x)[1]
        #print(x)
        batch, channel, height, width = x.size()
        #print('batch, channel, height, width')
        #print(batch, channel, height, width)
        x = x.view(batch, channel * height, width)
        x = x.permute(2, 0, 1)  # (width, batch, feature)
        #print(x.size())
        seq = self.map_to_seq(x)

        recurrent, _ = self.rnn1(seq)
        recurrent, _ = self.rnn2(recurrent)

        output = self.dense(recurrent)
        return output  # shape: (seq_len, batch, num_class)
        

    def _compute_output_shape(self,img_channel,  img_height, img_width):
        dummy_input = torch.randn(1, img_channel, img_height, img_width)
        dummy_output = self.model(dummy_input)
        #print(len(dummy_output))
        
        _, channels, height, width = dummy_output[1].size()
        #print('hihi')
        #print(channels, height, width)
        return channels, height, width

In [16]:
num_class = len(Synth90kDataset.LABEL2CHAR) + 1
model = MobileVit_improv(1,32,100,num_class)
#model.load_state_dict(torch.load('D:/lab2/深度学习课程设计/crnn-pytorch-master/checkpoints/mobileVit_chars74k_2.pth'),strict=False)
dummy_input = torch.randn(5, 1, 32, 100)
dummy_output =model(dummy_input)
dummy_output.size()

torch.Size([25, 5, 512])


torch.Size([25, 5, 37])

In [20]:

epochs = config['epochs']
train_batch_size = config['train_batch_size']
eval_batch_size = config['eval_batch_size']
lr = config['lr']
show_interval = config['show_interval']
valid_interval = config['valid_interval']
save_interval = config['save_interval']
end_interval = 40000
cpu_workers = config['cpu_workers']
reload_checkpoint = config['reload_checkpoint']
valid_max_iter = config['valid_max_iter']

img_width = config['img_width']
img_height = config['img_height']
img_channel = config['img_channel']
data_dir = config['data_dir']
#print(img_width,img_height)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'device: {device}')

train_dataset = Synth90kDataset(root_dir=data_dir, mode='train',
                                img_height=img_height, img_width=img_width)
valid_dataset = Synth90kDataset(root_dir=data_dir, mode='dev',
                                img_height=img_height, img_width=img_width)

print(valid_dataset.__len__())
train_loader = DataLoader(
    dataset=train_dataset,
    batch_size=train_batch_size,
    shuffle=True,
    num_workers=0,
    collate_fn=synth90k_collate_fn)
valid_loader = DataLoader(
    dataset=valid_dataset,
    batch_size=eval_batch_size,
    shuffle=True,
    num_workers=0,
    collate_fn=synth90k_collate_fn)

num_class = len(Synth90kDataset.LABEL2CHAR) + 1
'''
crnn = CRNN(1, img_height, img_width, num_class,
            map_to_seq_hidden=config['map_to_seq_hidden'],
            rnn_hidden=config['rnn_hidden'],
            leaky_relu=config['leaky_relu'])
if reload_checkpoint:
    crnn.load_state_dict(torch.load(reload_checkpoint, map_location=device))
'''  
    
crnn = MobileVit_improv(1,32,100,num_class)
#crnn.load_state_dict(torch.load('D:/lab2/深度学习课程设计/crnn-pytorch-master/checkpoints/mobileVit_chars74k_2.pth'),strict=False)


crnn.to(device)

optimizer = optim.Adam(crnn.parameters())
criterion = CTCLoss(reduction='sum', zero_infinity=True)
criterion.to(device)
loss_history=[]
valid_history=[]
assert save_interval % valid_interval == 0
i = 1
for epoch in range(1, epochs + 1):
    print(f'epoch: {epoch}')
    tot_train_loss = 0.
    tot_train_count = 0
    for train_data in train_loader:
        loss = train_batch(crnn, train_data, optimizer, criterion, device)
        train_size = train_data[0].size(0)

        tot_train_loss += loss
        tot_train_count += train_size
        if i % show_interval == 0:
            print('train_batch_loss[', i, ']: ', loss / train_size)
            loss_history.append(loss / train_size)

        if i % valid_interval == 0:
            evaluation = evaluate(crnn, valid_loader, criterion,
                                  decode_method=config['decode_method'],
                                  beam_size=config['beam_size'])
            print('valid_evaluation: loss={loss}, acc={acc}'.format(**evaluation))
            valid_history.append(evaluation)
            if i % save_interval == 0:
                prefix = 'mobilevit_rnn_v2'
                loss = evaluation['loss']
                save_model_path = os.path.join(config['checkpoints_dir'],
                                               f'{prefix}_{i:06}_loss{loss}.pt')
                torch.save(crnn.state_dict(), save_model_path)
                print('save model at ', save_model_path)

        if i % end_interval == 0:
            break
        i += 1

    print('train_loss: ', tot_train_loss / tot_train_count)

device: cuda
802734
epoch: 1
train_batch_loss[ 10 ]:  27.554855346679688
train_batch_loss[ 20 ]:  25.780851364135742
train_batch_loss[ 30 ]:  25.15695571899414
train_batch_loss[ 40 ]:  25.935163497924805
train_batch_loss[ 50 ]:  24.611080169677734
train_batch_loss[ 60 ]:  26.569761276245117
train_batch_loss[ 70 ]:  25.440114974975586
train_batch_loss[ 80 ]:  24.095073699951172
train_batch_loss[ 90 ]:  24.965740203857422
train_batch_loss[ 100 ]:  24.04668426513672
train_batch_loss[ 110 ]:  25.298580169677734
train_batch_loss[ 120 ]:  23.279333114624023
train_batch_loss[ 130 ]:  25.71746826171875
train_batch_loss[ 140 ]:  23.87799072265625
train_batch_loss[ 150 ]:  24.745609283447266
train_batch_loss[ 160 ]:  23.414257049560547
train_batch_loss[ 170 ]:  24.748353958129883
train_batch_loss[ 180 ]:  26.4576416015625
train_batch_loss[ 190 ]:  24.68633270263672
train_batch_loss[ 200 ]:  23.498371124267578
train_batch_loss[ 210 ]:  26.11236572265625
train_batch_loss[ 220 ]:  24.86026763916015

train_batch_loss[ 1840 ]:  4.224534034729004
train_batch_loss[ 1850 ]:  5.8397040367126465
train_batch_loss[ 1860 ]:  5.246426582336426
train_batch_loss[ 1870 ]:  6.962961196899414
train_batch_loss[ 1880 ]:  5.629364967346191
train_batch_loss[ 1890 ]:  8.11369514465332
train_batch_loss[ 1900 ]:  7.607270240783691
train_batch_loss[ 1910 ]:  5.176774978637695
train_batch_loss[ 1920 ]:  6.600076675415039
train_batch_loss[ 1930 ]:  6.745774269104004
train_batch_loss[ 1940 ]:  6.736330509185791
train_batch_loss[ 1950 ]:  4.695995330810547
train_batch_loss[ 1960 ]:  6.261013031005859
train_batch_loss[ 1970 ]:  5.013923645019531
train_batch_loss[ 1980 ]:  6.037240505218506
train_batch_loss[ 1990 ]:  3.889737606048584
train_batch_loss[ 2000 ]:  6.012662887573242
train_batch_loss[ 2010 ]:  5.808933258056641
train_batch_loss[ 2020 ]:  6.453112602233887
train_batch_loss[ 2030 ]:  6.6321516036987305
train_batch_loss[ 2040 ]:  6.145816326141357
train_batch_loss[ 2050 ]:  6.223575592041016
train_bat

train_batch_loss[ 3650 ]:  5.259361267089844
train_batch_loss[ 3660 ]:  4.31160831451416
train_batch_loss[ 3670 ]:  4.430137634277344
train_batch_loss[ 3680 ]:  4.143223285675049
train_batch_loss[ 3690 ]:  3.4304604530334473
train_batch_loss[ 3700 ]:  5.000253200531006
train_batch_loss[ 3710 ]:  6.214939117431641
train_batch_loss[ 3720 ]:  4.4908318519592285
train_batch_loss[ 3730 ]:  4.182753562927246
train_batch_loss[ 3740 ]:  3.543653964996338
train_batch_loss[ 3750 ]:  3.5838074684143066
train_batch_loss[ 3760 ]:  4.597156524658203
train_batch_loss[ 3770 ]:  4.551187992095947
train_batch_loss[ 3780 ]:  3.0707640647888184
train_batch_loss[ 3790 ]:  5.127890586853027
train_batch_loss[ 3800 ]:  3.7435190677642822
train_batch_loss[ 3810 ]:  2.4017493724823
train_batch_loss[ 3820 ]:  2.950615167617798
train_batch_loss[ 3830 ]:  4.386658191680908
train_batch_loss[ 3840 ]:  3.2031984329223633
train_batch_loss[ 3850 ]:  3.540591239929199
train_batch_loss[ 3860 ]:  4.437498092651367
train_b

Evaluate:  23%|████████████████                                                     | 365/1568 [05:19<16:27,  1.22it/s]

Corrupted image for 620308


...

Evaluate:  54%|█████████████████████████████████████▏                               | 846/1568 [10:46<08:59,  1.34it/s]

Corrupted image for 620308


Evaluate: 100%|████████████████████████████████████████████████████████████████████| 1568/1568 [19:55<00:00,  1.31it/s]


valid_evaluation: loss=3.343903567589988, acc=0.6249965742076453
save model at  D:/lab2/深度学习课程设计/crnn-pytorch-master/checkpoints/mobilevit_rnn_v2_008000_loss3.343903567589988.pt
train_batch_loss[ 8010 ]:  2.556375503540039
train_batch_loss[ 8020 ]:  3.3300390243530273
train_batch_loss[ 8030 ]:  3.0454397201538086
train_batch_loss[ 8040 ]:  5.355524063110352
train_batch_loss[ 8050 ]:  3.520481824874878
train_batch_loss[ 8060 ]:  3.9062724113464355
train_batch_loss[ 8070 ]:  2.982847213745117
train_batch_loss[ 8080 ]:  3.831601858139038
train_batch_loss[ 8090 ]:  3.3890154361724854
train_batch_loss[ 8100 ]:  4.26798677444458
train_batch_loss[ 8110 ]:  4.712113380432129
train_batch_loss[ 8120 ]:  3.913109064102173
train_batch_loss[ 8130 ]:  3.166780471801758
train_batch_loss[ 8140 ]:  4.569340229034424
train_batch_loss[ 8150 ]:  4.004302024841309
train_batch_loss[ 8160 ]:  4.250143527984619
train_batch_loss[ 8170 ]:  3.266453266143799
train_batch_loss[ 8180 ]:  2.507626533508301
train_bat

KeyboardInterrupt: 

In [12]:

import os
import pickle

 
with open('D:/lab2/深度学习课程设计/crnn-pytorch-master/train_history/mobileVit_rnn_loss', 'wb') as fp:
    pickle.dump([loss_history,valid_history], fp)

MemoryError: 

In [ ]:
with open ('D:/lab2/深度学习课程设计/crnn-pytorch-master/train_history/mobileVit_rnn_1', 'rb') as fp:
    temp = pickle.load(fp)
temp

In [ ]:
pytorch_total_params = sum(p.numel() for p in crnn.parameters())
pytorch_total_params

In [13]:
loss_history

[24.15996551513672,
 23.147846221923828,
 25.26816177368164,
 23.757020950317383,
 22.980716705322266,
 24.600717544555664,
 23.73973274230957,
 23.625492095947266,
 25.49618911743164,
 23.96426010131836,
 23.72730255126953,
 23.053531646728516,
 24.715412139892578,
 24.980304718017578,
 24.164417266845703,
 24.762134552001953,
 22.56833839416504,
 23.74498748779297,
 23.180675506591797,
 23.6967830657959,
 22.54412841796875,
 23.37295150756836,
 21.49481773376465,
 23.705638885498047,
 20.418201446533203,
 23.384031295776367,
 21.797086715698242,
 22.812301635742188,
 20.524314880371094,
 21.212459564208984,
 23.029254913330078,
 21.234966278076172,
 21.494609832763672,
 21.280935287475586,
 22.542755126953125,
 21.828720092773438,
 21.478775024414062,
 19.28566551208496,
 20.738784790039062,
 20.330886840820312,
 20.08675765991211,
 17.75045394897461,
 18.13392448425293,
 18.938695907592773,
 20.794984817504883,
 18.07672119140625,
 18.164703369140625,
 18.431640625,
 15.343786239624

# mobileVit special

In [13]:
class MobileViT_RNN(nn.Module):
    def __init__(self, image_size, dims, channels, num_classes, expansion=4, kernel_size=3, patch_size=(2, 2)):
        super().__init__()
        ih, iw = image_size
        ph, pw = patch_size
        assert ih % ph == 0 and iw % pw == 0

        L = [2, 4, 3]
        rnn_hidden = 256
        self.conv1 = conv_nxn_bn(1, channels[0], stride=2)
                
        '0    1   2     3   4     5    6    7'      
        '[16, 64, 128, 128, 256, 256, 512,512]'
            
        self.mv2 = nn.ModuleList([])
        self.mv2.append(MV2Block(channels[0], channels[1], 1, expansion)) #0
        self.mv2.append(MV2Block(channels[1], channels[2], 2, expansion)) #1
        #vit
        self.mv2.append(MV2Block(channels[3], channels[4], 2, expansion)) #2
        #vit
        self.mv2.append(MV2Block(channels[5], channels[6], 2, expansion)) #3
        #vit
        
        self.mvit = nn.ModuleList([])
        self.mvit.append(MobileViTBlock(dims[0], L[0], channels[2], kernel_size, patch_size, int(dims[0]*2)))
        self.mvit.append(MobileViTBlock(dims[1], L[1], channels[4], kernel_size, patch_size, int(dims[1]*4)))
        self.mvit.append(MobileViTBlock(dims[2], L[2], channels[6], kernel_size, patch_size, int(dims[2]*4)))

        self.conv2 = conv_1x1_bn(channels[-2], channels[-1])

        self.pool = nn.AvgPool2d(ih//32, 1)
        self.fc = nn.Linear(channels[-1], num_classes, bias=False)
        
        map_to_seq_hidden = 1000
        #self.model.head.fc = nn.Linear(self.model.head.in_features, map_to_seq)
        output_channel, output_height, output_width = self._compute_output_shape(1, image_size[1], image_size[0])
        
        self.map_to_seq = nn.Linear(output_channel * output_height, map_to_seq_hidden)
        self.rnn1 = nn.LSTM(map_to_seq_hidden, rnn_hidden, bidirectional=True)
        self.rnn2 = nn.LSTM(2 * rnn_hidden, rnn_hidden, bidirectional=True)

        self.dense = nn.Linear(2 * rnn_hidden, num_class)
        
    def backbone_forward(self,x):
        x = self.conv1(x)
        x = self.mv2[0](x)
        x = self.mv2[1](x)
        #print(x.shape)
        x = self.mvit[0](x)
        #print(x.shape)
        x = self.mv2[2](x)
        x = self.mvit[1](x)
        x = self.mv2[3](x)
        x = self.mvit[2](x)
        
        
        x = self.conv2(x)
        return x

    def forward(self, x):
        
        #x = self.pool(x).view(-1, x.shape[1])
        x = self.backbone_forward(x)
        batch, channel, height, width = x.size()
        #print('batch, channel, height, width')
        #print(batch, channel, height, width)
        x = x.view(batch, channel * height, width)
        x = x.permute(2, 0, 1)  # (width, batch, feature)
        #print(x.size())
        seq = self.map_to_seq(x)

        recurrent, _ = self.rnn1(seq)
        recurrent, _ = self.rnn2(recurrent)

        output = self.dense(recurrent)
        #x = self.fc(x)
        return output
    
    
    def _compute_output_shape(self,img_channels,  img_width, img_height):
        #print(img_channels,  img_height, img_width)
        dummy_input = torch.randn(1, img_channels, img_width, img_height)
        dummy_output = self.backbone_forward(dummy_input)
        #print(dummy_output.size())
        
        _, channels, width, height = dummy_output.size()
        
        return channels, height, width

In [14]:
num_class = 37
dims = [64, 80, 96]
channels = [16, 32, 64, 64, 128, 128, 256,256]
model = MobileViT_RNN((32, 128), dims, channels, num_classes = num_class, expansion=2)

dummy_input = torch.randn(5, 1, 32, 128)
dummy_output =model(dummy_input)
dummy_output.size()

torch.Size([1, 64, 32, 8])
torch.Size([1, 64, 32, 8])
torch.Size([5, 64, 8, 32])
torch.Size([5, 64, 8, 32])


torch.Size([8, 5, 37])

In [63]:
pytorch_total_params = sum(p.numel() for p in model.parameters())
pytorch_total_params

7862269

In [58]:

epochs = config['epochs']
train_batch_size = config['train_batch_size']
eval_batch_size = config['eval_batch_size']
lr = config['lr']
show_interval = config['show_interval']
valid_interval = config['valid_interval']
save_interval = config['save_interval']
end_interval = 40000
cpu_workers = config['cpu_workers']
reload_checkpoint = config['reload_checkpoint']
valid_max_iter = config['valid_max_iter']

img_width = 128
img_height = 32
img_channel = config['img_channel']
data_dir = config['data_dir']
#print(img_width,img_height)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'device: {device}')

train_dataset = Synth90kDataset(root_dir=data_dir, mode='train',
                                img_height=img_height, img_width=img_width)
valid_dataset = Synth90kDataset(root_dir=data_dir, mode='dev',
                                img_height=img_height, img_width=img_width)

print(valid_dataset.__len__())
train_loader = DataLoader(
    dataset=train_dataset,
    batch_size=train_batch_size,
    shuffle=True,
    num_workers=0,
    collate_fn=synth90k_collate_fn)
valid_loader = DataLoader(
    dataset=valid_dataset,
    batch_size=eval_batch_size,
    shuffle=True,
    num_workers=0,
    collate_fn=synth90k_collate_fn)

num_class = len(Synth90kDataset.LABEL2CHAR) + 1
'''
crnn = CRNN(1, img_height, img_width, num_class,
            map_to_seq_hidden=config['map_to_seq_hidden'],
            rnn_hidden=config['rnn_hidden'],
            leaky_relu=config['leaky_relu'])
if reload_checkpoint:
    crnn.load_state_dict(torch.load(reload_checkpoint, map_location=device))
'''  
dims = [64, 80, 96]
channels = [16, 32, 64, 64, 128, 128, 256,256]
crnn = MobileViT_RNN((32, 128), dims, channels, num_classes = num_class, expansion=2)
#crnn.load_state_dict(torch.load('D:/lab2/深度学习课程设计/crnn-pytorch-master/checkpoints/mobileVit_chars74k_2.pth'),strict=False)
pytorch_total_params = sum(p.numel() for p in crnn.parameters())
print('total param: ', pytorch_total_params)


device: cuda
802734
total param:  7862269


In [59]:

crnn.to(device)

optimizer = optim.Adam(crnn.parameters())
criterion = CTCLoss(reduction='sum', zero_infinity=True)
criterion.to(device)
loss_history=[]
valid_history=[]
assert save_interval % valid_interval == 0
i = 1
for epoch in range(1, epochs + 1):
    print(f'epoch: {epoch}')
    tot_train_loss = 0.
    tot_train_count = 0
    for train_data in train_loader:
        loss = train_batch(crnn, train_data, optimizer, criterion, device)
        train_size = train_data[0].size(0)

        tot_train_loss += loss
        tot_train_count += train_size
        if i % show_interval == 0:
            print('train_batch_loss[', i, ']: ', loss / train_size)
            loss_history.append(loss / train_size)

        if i % valid_interval == 0:
            evaluation = evaluate(crnn, valid_loader, criterion,
                                  decode_method=config['decode_method'],
                                  beam_size=config['beam_size'])
            print('valid_evaluation: loss={loss}, acc={acc}'.format(**evaluation))
            valid_history.append(evaluation)
            if i % save_interval == 0:
                prefix = 'mobilevit_rnn_v3'
                loss = evaluation['loss']
                save_model_path = os.path.join(config['checkpoints_dir'],
                                               f'{prefix}_{i:06}_loss{loss}.pt')
                torch.save(crnn.state_dict(), save_model_path)
                print('save model at ', save_model_path)

        if i % end_interval == 0:
            break
        i += 1

    print('train_loss: ', tot_train_loss / tot_train_count)

epoch: 1
train_batch_loss[ 10 ]:  9.833599090576172
train_batch_loss[ 20 ]:  10.535752296447754
train_batch_loss[ 30 ]:  10.780181884765625
train_batch_loss[ 40 ]:  11.867664337158203
train_batch_loss[ 50 ]:  10.562088012695312
train_batch_loss[ 60 ]:  10.592382431030273
train_batch_loss[ 70 ]:  9.933206558227539
train_batch_loss[ 80 ]:  10.512938499450684
train_batch_loss[ 90 ]:  11.573403358459473
train_batch_loss[ 100 ]:  11.273148536682129
train_batch_loss[ 110 ]:  10.757099151611328
train_batch_loss[ 120 ]:  10.757181167602539
train_batch_loss[ 130 ]:  10.68504524230957
train_batch_loss[ 140 ]:  10.932250022888184
train_batch_loss[ 150 ]:  11.772577285766602
train_batch_loss[ 160 ]:  13.45754623413086
train_batch_loss[ 170 ]:  11.529508590698242
train_batch_loss[ 180 ]:  11.235370635986328
train_batch_loss[ 190 ]:  9.458175659179688
train_batch_loss[ 200 ]:  9.761530876159668
train_batch_loss[ 210 ]:  10.659767150878906
train_batch_loss[ 220 ]:  12.334430694580078
train_batch_loss

train_batch_loss[ 1850 ]:  5.899908542633057
train_batch_loss[ 1860 ]:  4.720038414001465
train_batch_loss[ 1870 ]:  5.312541961669922
train_batch_loss[ 1880 ]:  4.1861772537231445
train_batch_loss[ 1890 ]:  4.794222831726074
train_batch_loss[ 1900 ]:  3.7928197383880615
train_batch_loss[ 1910 ]:  3.4472274780273438
train_batch_loss[ 1920 ]:  4.141385555267334
train_batch_loss[ 1930 ]:  3.939920425415039
train_batch_loss[ 1940 ]:  4.447235107421875
train_batch_loss[ 1950 ]:  3.0109336376190186
train_batch_loss[ 1960 ]:  3.002383232116699
train_batch_loss[ 1970 ]:  4.424862861633301
train_batch_loss[ 1980 ]:  4.445396900177002
train_batch_loss[ 1990 ]:  5.779014587402344
train_batch_loss[ 2000 ]:  4.895319938659668
train_batch_loss[ 2010 ]:  4.481448650360107
train_batch_loss[ 2020 ]:  5.327464580535889
train_batch_loss[ 2030 ]:  5.438371658325195
train_batch_loss[ 2040 ]:  3.8320236206054688
train_batch_loss[ 2050 ]:  5.733742713928223
train_batch_loss[ 2060 ]:  4.855203628540039
train

train_batch_loss[ 3660 ]:  3.078932285308838
train_batch_loss[ 3670 ]:  2.793851852416992
train_batch_loss[ 3680 ]:  3.736936092376709
train_batch_loss[ 3690 ]:  3.4963746070861816
train_batch_loss[ 3700 ]:  3.0187976360321045
train_batch_loss[ 3710 ]:  3.6988906860351562
train_batch_loss[ 3720 ]:  3.408034324645996
train_batch_loss[ 3730 ]:  3.9663617610931396
train_batch_loss[ 3740 ]:  2.047365427017212
train_batch_loss[ 3750 ]:  3.0721211433410645
train_batch_loss[ 3760 ]:  3.6721999645233154
train_batch_loss[ 3770 ]:  3.5331454277038574
train_batch_loss[ 3780 ]:  3.788383960723877
train_batch_loss[ 3790 ]:  3.5295610427856445
train_batch_loss[ 3800 ]:  3.5023210048675537
train_batch_loss[ 3810 ]:  3.9890472888946533
train_batch_loss[ 3820 ]:  2.0893845558166504
train_batch_loss[ 3830 ]:  2.6752681732177734
train_batch_loss[ 3840 ]:  4.110867500305176
train_batch_loss[ 3850 ]:  3.214167833328247
train_batch_loss[ 3860 ]:  3.4345438480377197
train_batch_loss[ 3870 ]:  2.8713479042053

Evaluate:   2%|█                                                                     | 25/1568 [00:21<19:21,  1.33it/s]

Corrupted image for 419552


...

Evaluate:  40%|███████████████████████████▋                                         | 629/1568 [06:35<09:35,  1.63it/s]

Corrupted image for 419552


Evaluate: 100%|████████████████████████████████████████████████████████████████████| 1568/1568 [16:17<00:00,  1.60it/s]


valid_evaluation: loss=2.6722909843792433, acc=0.2055089232547768
save model at  D:/lab2/深度学习课程设计/crnn-pytorch-master/checkpoints/mobilevit_rnn_v3_008000_loss2.6722909843792433.pt
train_batch_loss[ 8010 ]:  2.2248923778533936
train_batch_loss[ 8020 ]:  3.2494008541107178
train_batch_loss[ 8030 ]:  2.0741119384765625
train_batch_loss[ 8040 ]:  2.664968252182007


KeyboardInterrupt: 

In [ ]:
from torchvision import models
model = models.resnet50(pretrained=False)

In [ ]:
import timm

# from timm
pretrained_model_name = "resnet50"
model = timm.create_model(pretrained_model_name, pretrained=False)

In [26]:
import torch
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as transforms

vgg16 = models.vgg16(weights=False)
vgg16

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [ ]:
import torch.nn as nn


class CRNN(nn.Module):

    def __init__(self, img_channel, img_height, img_width, num_class,
                 map_to_seq_hidden=64, rnn_hidden=256, leaky_relu=False):
        super(CRNN, self).__init__()

        self.cnn, (output_channel, output_height, output_width) = \
            self._cnn_backbone(img_channel, img_height, img_width, leaky_relu)

        self.map_to_seq = nn.Linear(output_channel * output_height, map_to_seq_hidden)

        self.rnn1 = nn.LSTM(map_to_seq_hidden, rnn_hidden, bidirectional=True)
        self.rnn2 = nn.LSTM(2 * rnn_hidden, rnn_hidden, bidirectional=True)

        self.dense = nn.Linear(2 * rnn_hidden, num_class)

    def _cnn_backbone(self, img_channel, img_height, img_width, leaky_relu):
        assert img_height % 16 == 0
        assert img_width % 4 == 0

        channels = [img_channel, 64, 128, 256, 256, 512, 512, 512]
        kernel_sizes = [3, 3, 3, 3, 3, 3, 2]
        strides = [1, 1, 1, 1, 1, 1, 1]
        paddings = [1, 1, 1, 1, 1, 1, 0]

        cnn = nn.Sequential()

        def conv_relu(i, batch_norm=False):
            # shape of input: (batch, input_channel, height, width)
            input_channel = channels[i]
            output_channel = channels[i+1]

            cnn.add_module(
                f'conv{i}',
                nn.Conv2d(input_channel, output_channel, kernel_sizes[i], strides[i], paddings[i])
            )

            if batch_norm:
                cnn.add_module(f'batchnorm{i}', nn.BatchNorm2d(output_channel))

            relu = nn.LeakyReLU(0.2, inplace=True) if leaky_relu else nn.ReLU(inplace=True)
            cnn.add_module(f'relu{i}', relu)

        # size of image: (channel, height, width) = (img_channel, img_height, img_width)
        conv_relu(0)
        cnn.add_module('pooling0', nn.MaxPool2d(kernel_size=2, stride=2))
        # (64, img_height // 2, img_width // 2)

        conv_relu(1)
        cnn.add_module('pooling1', nn.MaxPool2d(kernel_size=2, stride=2))
        # (128, img_height // 4, img_width // 4)

        conv_relu(2)
        conv_relu(3)
        cnn.add_module(
            'pooling2',
            nn.MaxPool2d(kernel_size=(2, 1))
        )  # (256, img_height // 8, img_width // 4)

        conv_relu(4, batch_norm=True)
        conv_relu(5, batch_norm=True)
        cnn.add_module(
            'pooling3',
            nn.MaxPool2d(kernel_size=(2, 1))
        )  # (512, img_height // 16, img_width // 4)

        conv_relu(6)  # (512, img_height // 16 - 1, img_width // 4 - 1)

        output_channel, output_height, output_width = \
            channels[-1], img_height // 16 - 1, img_width // 4 - 1
        return cnn, (output_channel, output_height, output_width)

    def forward(self, images):
        # shape of images: (batch, channel, height, width)

        conv = self.cnn(images)
        batch, channel, height, width = conv.size()

        conv = conv.view(batch, channel * height, width)
        conv = conv.permute(2, 0, 1)  # (width, batch, feature)
        seq = self.map_to_seq(conv)

        recurrent, _ = self.rnn1(seq)
        recurrent, _ = self.rnn2(recurrent)

        output = self.dense(recurrent)
        return output  # shape: (seq_len, batch, num_class)

In [25]:
import torch
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as transforms



# Freeze the parameters of the pre-trained layers


class VGG_RNN(nn.Module):

    def __init__(self, img_channel, img_height, img_width, num_class,
                 map_to_seq_hidden=64, rnn_hidden=256, leaky_relu=False):
        super(VGG_RNN, self).__init__()
        vgg16 = models.vgg16(weights=False)
        
        
        vgg16.features[0] = nn.Conv2d(img_channel, 64, kernel_size=3, stride=1, padding=1)
        ''' for param in vgg16.features.parameters():
            param.requires_grad = False'''
        self.cnn  = vgg16.features
            
        
        output_channel, output_height, output_width = self._compute_output_shape(img_channel, img_height, img_width)
        
        self.map_to_seq = nn.Linear(output_channel * output_height, map_to_seq_hidden)

        self.rnn1 = nn.LSTM(map_to_seq_hidden, rnn_hidden, bidirectional=True)
        self.rnn2 = nn.LSTM(2 * rnn_hidden, rnn_hidden, bidirectional=True)

        self.dense = nn.Linear(2 * rnn_hidden, num_class)

    def _compute_output_shape(self,img_channel,  img_height, img_width):
        dummy_input = torch.randn(1, img_channel, img_height, img_width)
        dummy_output = self.cnn(dummy_input)
        _, channels, height, width = dummy_output.size()
        return channels, height, width

    def forward(self, images):
        # shape of images: (batch, channel, height, width)

        conv = self.cnn(images)
        batch, channel, height, width = conv.size()
        print('batch, channel, height, width' , batch, channel, height, width)
        conv = conv.view(batch, channel * height, width)
        conv = conv.permute(2, 0, 1)  # (width, batch, feature)
        seq = self.map_to_seq(conv)

        recurrent, _ = self.rnn1(seq)
        recurrent, _ = self.rnn2(recurrent)

        output = self.dense(recurrent)
        return output  # shape: (seq_len, batch, num_class)


In [26]:
def train_batch(crnn, data, optimizer, criterion, device):
    crnn.train()
    images, targets, target_lengths = [d.to(device) for d in data]
    print(images.size())
    logits = crnn(images)
    print(logits.size())
    log_probs = torch.nn.functional.log_softmax(logits, dim=2)

    batch_size = images.size(0)
    input_lengths = torch.LongTensor([logits.size(0)] * batch_size)
    target_lengths = torch.flatten(target_lengths)
    #print(batch_size)
    #print(input_lengths.size())
    #print(target_lengths.size())
    #print('end')
    loss = criterion(log_probs, targets, input_lengths, target_lengths)

    optimizer.zero_grad()
    loss.backward()
    torch.nn.utils.clip_grad_norm_(crnn.parameters(), 5) # gradient clipping with 5
    optimizer.step()
    return loss.item()



In [29]:


epochs = config['epochs']
train_batch_size = config['train_batch_size']
eval_batch_size = config['eval_batch_size']
lr = config['lr']
show_interval = config['show_interval']
valid_interval = config['valid_interval']
save_interval = config['save_interval']
cpu_workers = config['cpu_workers']
reload_checkpoint = config['reload_checkpoint']
valid_max_iter = config['valid_max_iter']

img_width = config['img_width']
img_height = config['img_height']
data_dir = config['data_dir']

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'device: {device}')

train_dataset = Synth90kDataset(root_dir=data_dir, mode='train',
                                img_height=img_height, img_width=img_width)
valid_dataset = Synth90kDataset(root_dir=data_dir, mode='dev',
                                img_height=img_height, img_width=img_width)

train_loader = DataLoader(
    dataset=train_dataset,
    batch_size=train_batch_size,
    shuffle=True,
    num_workers=0,
    collate_fn=synth90k_collate_fn)
valid_loader = DataLoader(
    dataset=valid_dataset,
    batch_size=eval_batch_size,
    shuffle=True,
    num_workers=0,
    collate_fn=synth90k_collate_fn)

num_class = len(Synth90kDataset.LABEL2CHAR) + 1
crnn = VGG_RNN(1, img_height, img_width, num_class,
            map_to_seq_hidden=config['map_to_seq_hidden'],
            rnn_hidden=config['rnn_hidden'],
            leaky_relu=config['leaky_relu'])
if reload_checkpoint:
    crnn.load_state_dict(torch.load(reload_checkpoint, map_location=device))
crnn.to(device)

optimizer = optim.Adam(crnn.parameters())
criterion = CTCLoss(reduction='sum', zero_infinity=True)
criterion.to(device)

assert save_interval % valid_interval == 0
i = 1
for epoch in range(1, epochs + 1):
    print(f'epoch: {epoch}')
    tot_train_loss = 0.
    tot_train_count = 0
    for train_data in train_loader:
        loss = train_batch(crnn, train_data, optimizer, criterion, device)
        train_size = train_data[0].size(0)

        tot_train_loss += loss
        tot_train_count += train_size
        if i % show_interval == 0:
            print('train_batch_loss[', i, ']: ', loss / train_size)

        if i % valid_interval == 0:
            evaluation = evaluate(crnn, valid_loader, criterion,
                                  decode_method=config['decode_method'],
                                  beam_size=config['beam_size'])
            print('valid_evaluation: loss={loss}, acc={acc}'.format(**evaluation))

            if i % save_interval == 0:
                prefix = 'crnn'
                loss = evaluation['loss']
                save_model_path = os.path.join(config['checkpoints_dir'],
                                               f'{prefix}_{i:06}_loss{loss}.pt')
                torch.save(crnn.state_dict(), save_model_path)
                print('save model at ', save_model_path)

        i += 1

    print('train_loss: ', tot_train_loss / tot_train_count)



device: cuda
epoch: 1
image size
torch.Size([64, 1, 64, 64])
batch, channel, height, width 64 512 2 2
batch logits log_probs targets input_len target_len
64
torch.Size([2, 64, 37])
torch.Size([2, 64, 37])
torch.Size([539])
torch.Size([64])
torch.Size([64])
end
image size
torch.Size([64, 1, 64, 64])
batch, channel, height, width 64 512 2 2
batch logits log_probs targets input_len target_len
64
torch.Size([2, 64, 37])
torch.Size([2, 64, 37])
torch.Size([533])
torch.Size([64])
torch.Size([64])
end
image size
torch.Size([64, 1, 64, 64])
batch, channel, height, width 64 512 2 2
batch logits log_probs targets input_len target_len
64
torch.Size([2, 64, 37])
torch.Size([2, 64, 37])
torch.Size([509])
torch.Size([64])
torch.Size([64])
end
image size
torch.Size([64, 1, 64, 64])
batch, channel, height, width 64 512 2 2
batch logits log_probs targets input_len target_len
64
torch.Size([2, 64, 37])
torch.Size([2, 64, 37])
torch.Size([533])
torch.Size([64])
torch.Size([64])
end
image size
torch.Size(

KeyboardInterrupt: 

In [11]:


epochs = config['epochs']
train_batch_size = config['train_batch_size']
eval_batch_size = config['eval_batch_size']
lr = config['lr']
show_interval = config['show_interval']
valid_interval = config['valid_interval']
save_interval = config['save_interval']
cpu_workers = config['cpu_workers']
reload_checkpoint = config['reload_checkpoint']
valid_max_iter = config['valid_max_iter']

img_width = config['img_width']
img_height = config['img_height']
data_dir = config['data_dir']

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'device: {device}')

train_dataset = Synth90kDataset(root_dir=data_dir, mode='train',
                                img_height=img_height, img_width=img_width)
valid_dataset = Synth90kDataset(root_dir=data_dir, mode='dev',
                                img_height=img_height, img_width=img_width)

train_loader = DataLoader(
    dataset=train_dataset,
    batch_size=train_batch_size,
    shuffle=True,
    num_workers=0,
    collate_fn=synth90k_collate_fn)
valid_loader = DataLoader(
    dataset=valid_dataset,
    batch_size=eval_batch_size,
    shuffle=True,
    num_workers=0,
    collate_fn=synth90k_collate_fn)

num_class = len(Synth90kDataset.LABEL2CHAR) + 1
crnn = VGG_RNN(1, img_height, img_width, num_class,
            map_to_seq_hidden=config['map_to_seq_hidden'],
            rnn_hidden=config['rnn_hidden'],
            leaky_relu=config['leaky_relu'])
if reload_checkpoint:
    crnn.load_state_dict(torch.load(reload_checkpoint, map_location=device))
crnn.to(device)

optimizer = optim.RMSprop(crnn.parameters(), lr=lr)
criterion = CTCLoss(reduction='sum', zero_infinity=True)
criterion.to(device)

assert save_interval % valid_interval == 0
history = []

i = 1
for epoch in range(1, epochs + 1):
    print(f'epoch: {epoch}')
    tot_train_loss = 0.
    tot_train_count = 0
    for train_data in train_loader:
        loss = train_batch(crnn, train_data, optimizer, criterion, device)
        train_size = train_data[0].size(0)

        tot_train_loss += loss
        tot_train_count += train_size
        if i % show_interval == 0:
            print('train_batch_loss[', i, ']: ', loss / train_size)
            history.append(loss / train_size)

        if i % valid_interval == 0:
            evaluation = evaluate(crnn, valid_loader, criterion,
                                  decode_method=config['decode_method'],
                                  beam_size=config['beam_size'])
            print('valid_evaluation: loss={loss}, acc={acc}'.format(**evaluation))

            if i % save_interval == 0:
                prefix = 'vgg-rnn'
                loss = evaluation['loss']
                save_model_path = os.path.join(config['checkpoints_dir'],
                                               f'{prefix}_{i:06}_loss{loss}.pt')
                torch.save(crnn.state_dict(), save_model_path)
                print('save model at ', save_model_path)

        i += 1

    print('train_loss: ', tot_train_loss / tot_train_count)



device: cuda
epoch: 1
train_batch_loss[ 10 ]:  0.19078294932842255
train_batch_loss[ 20 ]:  0.23420129716396332
train_batch_loss[ 30 ]:  0.3162612318992615
train_batch_loss[ 40 ]:  0.3073037564754486
train_batch_loss[ 50 ]:  0.22981135547161102
train_batch_loss[ 60 ]:  0.19790568947792053
train_batch_loss[ 70 ]:  0.28985390067100525
train_batch_loss[ 80 ]:  0.3370938003063202
train_batch_loss[ 90 ]:  0.17985296249389648
train_batch_loss[ 100 ]:  0.28941500186920166
train_batch_loss[ 110 ]:  0.17580735683441162
train_batch_loss[ 120 ]:  0.10407394170761108
train_batch_loss[ 130 ]:  0.10388155281543732
train_batch_loss[ 140 ]:  0.2338990867137909
train_batch_loss[ 150 ]:  0.1459864377975464
train_batch_loss[ 160 ]:  0.1163751482963562
train_batch_loss[ 170 ]:  0.045228131115436554
train_batch_loss[ 180 ]:  0.3173154294490814


KeyboardInterrupt: 

In [32]:
import os
import glob

import torch
from torch.utils.data import Dataset
from scipy import signal
from scipy.io import wavfile
import cv2
from PIL import Image
import numpy as np


class Synth90kDataset(Dataset):
    CHARS = '0123456789abcdefghijklmnopqrstuvwxyz'
    CHAR2LABEL = {char: i + 1 for i, char in enumerate(CHARS)}
    LABEL2CHAR = {label: char for char, label in CHAR2LABEL.items()}

    def __init__(self, root_dir=None, mode=None, paths=None, img_height=32, img_width=100,img_channel=3):
        if root_dir and mode and not paths:
            paths, texts = self._load_from_raw_files(root_dir, mode)
        elif not root_dir and not mode and paths:
            texts = None

        self.paths = paths
        self.texts = texts
        self.img_height = img_height
        self.img_width = img_width
        self.img_channel = img_channel

    def _load_from_raw_files(self, root_dir, mode):
        mapping = {}
        with open(os.path.join(root_dir, 'lexicon.txt'), 'r') as fr:
            for i, line in enumerate(fr.readlines()):
                mapping[i] = line.strip()

        paths_file = None
        if mode == 'train':
            paths_file = 'annotation_train.txt'
        elif mode == 'dev':
            paths_file = 'annotation_val.txt'
        elif mode == 'test':
            paths_file = 'annotation_test.txt'

        paths = []
        texts = []
        with open(os.path.join(root_dir, paths_file), 'r') as fr:
            for line in fr.readlines():
                path, index_str = line.strip().split(' ')
                path = os.path.join(root_dir, path)
                index = int(index_str)
                text = mapping[index]
                paths.append(path)
                texts.append(text)
        return paths, texts

    def __len__(self):
        return len(self.paths)

    def __getitem__(self, index):
        path = self.paths[index]

        try:
            image = Image.open(path).convert('L')  # grey-scale
        except IOError:
            print('Corrupted image for %d' % index)
            return self[index + 1]
        image = image.convert(mode='RGB')
        
        image = image.resize((self.img_width, self.img_height), resample=Image.BILINEAR)
        image = np.array(image)
        
        image = image.reshape((self.img_channel,self.img_height, self.img_width ))
        image = (image / 127.5) - 1.0

        image = torch.FloatTensor(image)
        #print(image.shape)
        if self.texts:
            text = self.texts[index]
            target = [self.CHAR2LABEL[c] for c in text]
            target_length = [len(target)]

            target = torch.LongTensor(target)
            target_length = torch.LongTensor(target_length)
            return image, target, target_length
        else:
            return image


def synth90k_collate_fn(batch):
    images, targets, target_lengths = zip(*batch)
    images = torch.stack(images, 0)
    targets = torch.cat(targets, 0)
    target_lengths = torch.cat(target_lengths, 0)
    return images, targets, target_lengths


In [36]:
def train_batch(crnn, data, optimizer, criterion, device):
    crnn.train()
    images, targets, target_lengths = [d.to(device) for d in data]
    #print(images.size())
    logits = crnn(images)
    #print(logits.size())
    log_probs = torch.nn.functional.log_softmax(logits,dim=2)

    batch_size = images.size(0)
    input_lengths = torch.LongTensor([logits.size(0)] * batch_size)
    target_lengths = torch.flatten(target_lengths)

    loss = criterion(log_probs, targets, input_lengths, target_lengths)

    optimizer.zero_grad()
    loss.backward()
    torch.nn.utils.clip_grad_norm_(crnn.parameters(), 5) # gradient clipping with 5
    optimizer.step()
    return loss.item()



In [8]:
from einops import rearrange

def conv_1x1_bn(inp, oup):
    return nn.Sequential(
        nn.Conv2d(inp, oup, 1, 1, 0, bias=False),
        nn.BatchNorm2d(oup),
        nn.SiLU()
    )


def conv_nxn_bn(inp, oup, kernal_size=3, stride=1):
    return nn.Sequential(
        nn.Conv2d(inp, oup, kernal_size, stride, 1, bias=False),
        nn.BatchNorm2d(oup),
        nn.SiLU()
    )


class PreNorm(nn.Module):
    def __init__(self, dim, fn):
        super().__init__()
        self.norm = nn.LayerNorm(dim)
        self.fn = fn
    
    def forward(self, x, **kwargs):
        return self.fn(self.norm(x), **kwargs)


class FeedForward(nn.Module):
    def __init__(self, dim, hidden_dim, dropout=0.):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(dim, hidden_dim),
            nn.SiLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim, dim),
            nn.Dropout(dropout)
        )
    
    def forward(self, x):
        return self.net(x)


class Attention(nn.Module):
    def __init__(self, dim, heads=8, dim_head=64, dropout=0.):
        super().__init__()
        inner_dim = dim_head *  heads
        project_out = not (heads == 1 and dim_head == dim)

        self.heads = heads
        self.scale = dim_head ** -0.5

        self.attend = nn.Softmax(dim = -1)
        self.to_qkv = nn.Linear(dim, inner_dim * 3, bias = False)

        self.to_out = nn.Sequential(
            nn.Linear(inner_dim, dim),
            nn.Dropout(dropout)
        ) if project_out else nn.Identity()

    def forward(self, x):
        qkv = self.to_qkv(x).chunk(3, dim=-1)
        q, k, v = map(lambda t: rearrange(t, 'b p n (h d) -> b p h n d', h = self.heads), qkv)

        dots = torch.matmul(q, k.transpose(-1, -2)) * self.scale
        attn = self.attend(dots)
        out = torch.matmul(attn, v)
        out = rearrange(out, 'b p h n d -> b p n (h d)')
        return self.to_out(out)


class Transformer(nn.Module):
    def __init__(self, dim, depth, heads, dim_head, mlp_dim, dropout=0.):
        super().__init__()
        self.layers = nn.ModuleList([])
        for _ in range(depth):
            self.layers.append(nn.ModuleList([
                PreNorm(dim, Attention(dim, heads, dim_head, dropout)),
                PreNorm(dim, FeedForward(dim, mlp_dim, dropout))
            ]))
    
    def forward(self, x):
        for attn, ff in self.layers:
            x = attn(x) + x
            x = ff(x) + x
        return x


class MV2Block(nn.Module):
    def __init__(self, inp, oup, stride=1, expansion=4):
        super().__init__()
        self.stride = stride
        assert stride in [1, 2]

        hidden_dim = int(inp * expansion)
        self.use_res_connect = self.stride == 1 and inp == oup

        if expansion == 1:
            self.conv = nn.Sequential(
                # dw
                nn.Conv2d(hidden_dim, hidden_dim, 3, stride, 1, groups=hidden_dim, bias=False),
                nn.BatchNorm2d(hidden_dim),
                nn.SiLU(),
                # pw-linear
                nn.Conv2d(hidden_dim, oup, 1, 1, 0, bias=False),
                nn.BatchNorm2d(oup),
            )
        else:
            self.conv = nn.Sequential(
                # pw
                nn.Conv2d(inp, hidden_dim, 1, 1, 0, bias=False),
                nn.BatchNorm2d(hidden_dim),
                nn.SiLU(),
                # dw
                nn.Conv2d(hidden_dim, hidden_dim, 3, stride, 1, groups=hidden_dim, bias=False),
                nn.BatchNorm2d(hidden_dim),
                nn.SiLU(),
                # pw-linear
                nn.Conv2d(hidden_dim, oup, 1, 1, 0, bias=False),
                nn.BatchNorm2d(oup),
            )

    def forward(self, x):
        if self.use_res_connect:
            return x + self.conv(x)
        else:
            return self.conv(x)


class MobileViTBlock(nn.Module):
    def __init__(self, dim, depth, channel, kernel_size, patch_size, mlp_dim, dropout=0.):
        super().__init__()
        self.ph, self.pw = patch_size

        self.conv1 = conv_nxn_bn(channel, channel, kernel_size)
        self.conv2 = conv_1x1_bn(channel, dim)

        self.transformer = Transformer(dim, depth, 4, 8, mlp_dim, dropout)

        self.conv3 = conv_1x1_bn(dim, channel)
        self.conv4 = conv_nxn_bn(2 * channel, channel, kernel_size)
    
    def forward(self, x):
        y = x.clone()

        # Local representations
        x = self.conv1(x)
        x = self.conv2(x)
        
        # Global representations
        _, _, h, w = x.shape
        x = rearrange(x, 'b d (h ph) (w pw) -> b (ph pw) (h w) d', ph=self.ph, pw=self.pw)
        x = self.transformer(x)
        x = rearrange(x, 'b (ph pw) (h w) d -> b d (h ph) (w pw)', h=h//self.ph, w=w//self.pw, ph=self.ph, pw=self.pw)

        # Fusion
        x = self.conv3(x)
        x = torch.cat((x, y), 1)
        x = self.conv4(x)
        return x

In [5]:
class MobileVit_improv(nn.Module):
    def __init__(self, image_size, dims, channels, num_classes, expansion=4, kernel_size=3, patch_size=(2, 2)):
        super().__init__()
        ih, iw = image_size
        ph, pw = patch_size
        assert ih % ph == 0 and iw % pw == 0

        L = [2, 4, 3]
        rnn_hidden = 256
        self.conv1 = conv_nxn_bn(1, channels[0], stride=2)

        self.mv2 = nn.ModuleList([])
        self.mv2.append(MV2Block(channels[0], channels[1], 1, expansion))
        self.mv2.append(MV2Block(channels[1], channels[2], 1, expansion))
        self.mv2.append(MV2Block(channels[2], channels[3], 1, expansion))
        self.mv2.append(MV2Block(channels[2], channels[3], 1, expansion))   # Repeat
        self.mv2.append(MV2Block(channels[3], channels[4], 1, expansion))
        self.mv2.append(MV2Block(channels[5], channels[6], 2, expansion))
        self.mv2.append(MV2Block(channels[7], channels[8], 2, expansion))
        
        self.mvit = nn.ModuleList([])
        self.mvit.append(MobileViTBlock(dims[0], L[0], channels[5], kernel_size, patch_size, int(dims[0]*2)))
        self.mvit.append(MobileViTBlock(dims[1], L[1], channels[7], kernel_size, patch_size, int(dims[1]*4)))
        self.mvit.append(MobileViTBlock(dims[2], L[2], channels[9], kernel_size, patch_size, int(dims[2]*4)))

        self.conv2 = conv_1x1_bn(channels[-2], channels[-1])

        self.pool = nn.AvgPool2d(ih//32, 1)
        self.fc = nn.Linear(channels[-1], num_classes, bias=False)
        
        map_to_seq_hidden = 1000
        #self.model.head.fc = nn.Linear(self.model.head.in_features, map_to_seq)
        output_channel, output_height, output_width = self._compute_output_shape(1, image_size[1], image_size[0])
        
        self.map_to_seq = nn.Linear(output_channel * output_height, map_to_seq_hidden)
        self.rnn1 = nn.LSTM(map_to_seq_hidden, rnn_hidden, bidirectional=True)
        self.rnn2 = nn.LSTM(2 * rnn_hidden, rnn_hidden, bidirectional=True)

        self.dense = nn.Linear(2 * rnn_hidden, num_class)
        
    def backbone_forward(self,x):
        x = self.conv1(x)
        x = self.mv2[0](x)

        x = self.mv2[1](x)
        x = self.mv2[2](x)
        x = self.mv2[3](x)      # Repeat

        x = self.mv2[4](x)
        x = self.mvit[0](x)

        x = self.mv2[5](x)
        x = self.mvit[1](x)

        x = self.mv2[6](x)
        x = self.mvit[2](x)
        x = self.conv2(x)
        return x

    def forward(self, x):
        
        #x = self.pool(x).view(-1, x.shape[1])
        x = self.backbone_forward(x)
        batch, channel, height, width = x.size()
        print('batch, channel, height, width')
        print(batch, channel, height, width)
        x = x.view(batch, channel * height, width)
        x = x.permute(2, 0, 1)  # (width, batch, feature)
        #print(x.size())
        seq = self.map_to_seq(x)

        recurrent, _ = self.rnn1(seq)
        recurrent, _ = self.rnn2(recurrent)

        output = self.dense(recurrent)
        #x = self.fc(x)
        return output
    
    
    def _compute_output_shape(self,img_channels,  img_width, img_height):
        print(img_channels,  img_height, img_width)
        dummy_input = torch.randn(1, img_channels, img_width, img_height)
        dummy_output = self.backbone_forward(dummy_input)
        print(dummy_output.size())
        
        _, channels, width, height = dummy_output.size()
        
        return channels, height, width